<a href="https://colab.research.google.com/github/theamrzaki/ETL-Ensemble-at-Training-for-Low-cost-Open-source-Large-Language-Models-with-Multiple-Teachers/blob/main/Lmixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets==2.15.0   # Upgrade datasets to the latest version

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.10.0 which is incompatible.
torch 2.6.0+cu124

In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0929a8ae64ab5bf48a354b054186fd8ebcb441a20c7706f63c554affd24ac0e2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import pickle
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_metric
import torch
from torch.utils.tensorboard import SummaryWriter

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the data from the pickle file
with open("/content/drive/MyDrive/PhD/Research/Lmixer/supervised_data_with_attentions.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Create Dataset object
data = []
for example in loaded_data:
    data.append({
        "input_text": example["input_text"],
        "summary_t5": example["summary_t5"],
        "summary_bart": example["summary_bart"],
        "summary_gpt2": example["summary_gpt2"],
        "highlights": example["highlights"]
    })

dataset = Dataset.from_list(data)

# Format input function (concatenate all 3 summaries)
def format_input(summary_t5, summary_bart, summary_gpt2):
    return (
        f"Summary T5: {summary_t5} \n"
        f"Summary BART: {summary_bart} \n"
        f"Summary GPT-2: {summary_gpt2}"
    )

# Preprocessing function for tokenization
def preprocess_function(examples):
    input_texts = [
            format_input(example[0], example[1], example[2])
        for example in zip(examples["summary_t5"], examples["summary_bart"], examples["summary_gpt2"])
    ]

    target_texts = examples["highlights"]

    # Tokenizing inputs and outputs
    model_inputs = t5_tokenizer(input_texts, max_length=256, truncation=True, padding=True)
    labels = t5_tokenizer(target_texts, max_length=150, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load pre-trained T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load ROUGE metric
rouge_metric = load_metric("rouge")

# Compute ROUGE scores (custom metric)
def compute_rouge(p):
    predictions = p.predictions
    references = p.label_ids
    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_refs = t5_tokenizer.batch_decode(references, skip_special_tokens=True)

    # Use ROUGE-1 F-measure as the evaluation metric
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_refs)
    return {"rouge1": result["rouge1"].fmeasure, "rouge2": result["rouge2"].fmeasure, "rougeL": result["rougeL"].fmeasure}

# TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned",  # output directory for the model
    evaluation_strategy="no",  # evaluation strategy (after each epoch)
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=1,  # reduce batch size for training
    #per_device_eval_batch_size=1,  # reduce batch size for evaluation
    gradient_accumulation_steps=8,  # Accumulate gradients over 4 steps
    num_train_epochs=1,  # number of training epochs
    weight_decay=0.01,  # weight decay
    logging_dir="./logs",  # directory for logs
    logging_strategy="steps",  # log every few steps
    logging_steps=10,  # log every 500 steps
    report_to="tensorboard",  # report to TensorBoard
    fp16=True,  # Enable mixed precision training
        #eval_accumulation_steps=2, # Accumulate predictions during evaluation
)

# Setup TensorBoard
writer = SummaryWriter(log_dir="/content/drive/MyDrive/PhD/Research/Lmixer/logs")

# Trainer to handle the training loop
trainer = Trainer(
    model=t5_model,                   # the model to train
    args=training_args,               # training arguments
    train_dataset=tokenized_datasets, # training dataset
    #eval_dataset=tokenized_datasets,  # evaluation dataset
    #compute_metrics=compute_rouge,    # compute ROUGE during evaluation
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
t5_model.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned")
t5_tokenizer.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned")

# Close TensorBoard writer
writer.close()

print("Fine-tuning complete! Model saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<ipython-input-5-02b7fd87bad3>:60: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,12.805500
20,10.066200
30,8.197100
40,6.150100
50,4.658300
60,4.005700
70,3.233100
80,2.874500
90,2.780900
100,2.613300


Fine-tuning complete! Model saved.


In [1]:
import pickle
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_metric
import torch
from torch.utils.tensorboard import SummaryWriter

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the data from the pickle file
with open("/content/drive/MyDrive/PhD/Research/Lmixer/supervised_data_with_attentions.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Create Dataset object
data = []
for example in loaded_data:
    data.append({
        "input_text": example["input_text"],
        "summary_t5": example["summary_t5"],
        "summary_bart": example["summary_bart"],
        "summary_gpt2": example["summary_gpt2"],
        "highlights": example["highlights"]
    })

dataset = Dataset.from_list(data)

# Preprocessing function for baseline (using only input_text and highlights)
def preprocess_baseline_function(examples):
    input_texts = examples["input_text"]  # Only use input_text
    target_texts = examples["highlights"]  # Use highlights as the target

    # Tokenizing inputs and outputs
    model_inputs = t5_tokenizer(input_texts, max_length=512, truncation=True, padding=True)
    labels = t5_tokenizer(target_texts, max_length=150, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load pre-trained T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Apply preprocessing for baseline
tokenized_baseline_datasets = dataset.map(preprocess_baseline_function, batched=True)

# Load ROUGE metric
rouge_metric = load_metric("rouge")

# Compute ROUGE scores (custom metric)
def compute_rouge(p):
    predictions = p.predictions
    references = p.label_ids
    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_refs = t5_tokenizer.batch_decode(references, skip_special_tokens=True)

    # Use ROUGE-1 F-measure as the evaluation metric
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_refs)
    return {"rouge1": result["rouge1"].fmeasure, "rouge2": result["rouge2"].fmeasure, "rougeL": result["rougeL"].fmeasure}

# TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PhD/Research/Lmixer/finetuned_baseline",  # output directory for the model
    evaluation_strategy="no",  # evaluation strategy (after each epoch)
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=1,  # reduce batch size for training
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps
    num_train_epochs=1,  # number of training epochs
    weight_decay=0.01,  # weight decay
    logging_dir="./logs",  # directory for logs
    logging_strategy="steps",  # log every few steps
    logging_steps=10,  # log every 10 steps
    report_to="tensorboard",  # report to TensorBoard
    fp16=True,  # Enable mixed precision training
)

# Setup TensorBoard
writer = SummaryWriter(log_dir="/content/drive/MyDrive/PhD/Research/Lmixer/logs_baseline")

# Trainer to handle the training loop for baseline
trainer_baseline = Trainer(
    model=t5_model,                   # the model to train
    args=training_args,               # training arguments
    train_dataset=tokenized_baseline_datasets,  # training dataset for baseline
)

# Start fine-tuning for baseline model
trainer_baseline.train()

# Save the fine-tuned baseline model
t5_model.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/finetuned_baseline")
t5_tokenizer.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/finetuned_baseline")

# Close TensorBoard writer
writer.close()

print("Fine-tuning complete! Baseline model saved in 'finetuned_baseline'.")


ModuleNotFoundError: No module named 'datasets'

### Single Teacher

In [5]:
import pickle
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_metric
import torch
from torch.utils.tensorboard import SummaryWriter

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the data from the pickle file
with open("/content/drive/MyDrive/PhD/Research/Lmixer/supervised_data_with_attentions.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Create Dataset object
data = []
for example in loaded_data:
    data.append({
        "input_text": example["input_text"],
        "summary_t5": example["summary_t5"],
        "summary_bart": example["summary_bart"],
        "summary_gpt2": example["summary_gpt2"],
        "highlights": example["highlights"]
    })

dataset = Dataset.from_list(data)

# Format input function (concatenate all 3 summaries)
def format_input_single_teacher(summary_t5):
    return (
        f"Summary T5: {summary_t5} \n"
    )

# Preprocessing function for tokenization
def preprocess_function(examples):
    input_texts = [
            format_input_single_teacher(example[0])
        for example in zip(examples["summary_t5"])
    ]

    target_texts = examples["highlights"]

    # Tokenizing inputs and outputs
    model_inputs = t5_tokenizer(input_texts, max_length=256, truncation=True, padding=True)
    labels = t5_tokenizer(target_texts, max_length=150, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load pre-trained T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load ROUGE metric
rouge_metric = load_metric("rouge")

# Compute ROUGE scores (custom metric)
def compute_rouge(p):
    predictions = p.predictions
    references = p.label_ids
    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_refs = t5_tokenizer.batch_decode(references, skip_special_tokens=True)

    # Use ROUGE-1 F-measure as the evaluation metric
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_refs)
    return {"rouge1": result["rouge1"].fmeasure, "rouge2": result["rouge2"].fmeasure, "rougeL": result["rougeL"].fmeasure}

# TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned_single_teacher",  # output directory for the model
    evaluation_strategy="no",  # evaluation strategy (after each epoch)
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=1,  # reduce batch size for training
    #per_device_eval_batch_size=1,  # reduce batch size for evaluation
    gradient_accumulation_steps=8,  # Accumulate gradients over 4 steps
    num_train_epochs=1,  # number of training epochs
    weight_decay=0.01,  # weight decay
    logging_dir="./logs",  # directory for logs
    logging_strategy="steps",  # log every few steps
    logging_steps=10,  # log every 500 steps
    report_to="tensorboard",  # report to TensorBoard
    fp16=True,  # Enable mixed precision training
        #eval_accumulation_steps=2, # Accumulate predictions during evaluation
)

# Setup TensorBoard
writer = SummaryWriter(log_dir="/content/drive/MyDrive/PhD/Research/Lmixer/logs")

# Trainer to handle the training loop
trainer = Trainer(
    model=t5_model,                   # the model to train
    args=training_args,               # training arguments
    train_dataset=tokenized_datasets, # training dataset
    #eval_dataset=tokenized_datasets,  # evaluation dataset
    #compute_metrics=compute_rouge,    # compute ROUGE during evaluation
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
t5_model.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned_single_teacher")
t5_tokenizer.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned_single_teacher")

# Close TensorBoard writer
writer.close()

print("Fine-tuning complete! Model saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<ipython-input-5-9dac3e09ac22>:58: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,12.237100
20,9.499900
30,7.910500
40,6.201100
50,4.860700
60,4.219200
70,3.348500
80,2.957700
90,2.902700
100,2.668300


Fine-tuning complete! Model saved.


In [ ]:
#t5_model.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned")
#t5_tokenizer.save_pretrained("/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned")

('/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned/spiece.model',
 '/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned/added_tokens.json')

In [ ]:
t5_model

NameError: name 't5_model' is not defined

In [ ]:
from datasets import load_metric

## Evaluate the fine-tuned model

In [ ]:
import pickle
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_metric
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the data from the pickle file
#with open("/content/drive/MyDrive/PhD/Research/Lmixer/supervised_data_with_attentions.pkl", "rb") as f:
#    loaded_data = pickle.load(f)


# Load fine-tuned T5 model and tokenizer
model_path = "/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned"
t5_tokenizer_fine = T5Tokenizer.from_pretrained(model_path)
fine_tuned_model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)


model_path_baseline = "/content/drive/MyDrive/PhD/Research/Lmixer/finetuned_baseline"
t5_tokenizer_baseline = T5Tokenizer.from_pretrained(model_path_baseline)
fine_tuned_baseline_model = T5ForConditionalGeneration.from_pretrained(model_path_baseline).to(device)


# Create Dataset object (using only 100 sentences)
data = []
for i in range(2000, 2010):   # Take only the first 100 sentences
    example = loaded_data[i]
    data.append({
        "input_text": example["input_text"],
        "summary_t5": example["summary_t5"],
        "summary_bart": example["summary_bart"],
        "summary_gpt2": example["summary_gpt2"],
        "highlights": example["highlights"]
    })

dataset = Dataset.from_list(data)

# Format input function (concatenate all 3 summaries)
def format_input(summary_t5, summary_bart, summary_gpt2):
    return (
        f"Summary T5: {summary_t5} \n"
        f"Summary BART: {summary_bart} \n"
        f"Summary GPT-2: {summary_gpt2}"
    )

# Load pre-trained T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)  # Move to device

# Load fine-tuned T5 model
##fine_tuned_model = T5ForConditionalGeneration.from_pretrained("t5_finetuned_model").to(device) # Move to device


# Function to generate summaries and calculate ROUGE scores
def evaluate_model(model, tokenizer, dataset):
    rouge_metric = load_metric("rouge")

    all_predictions = []
    all_references = []

    for example in tqdm(dataset):
        input_text = format_input(example["summary_t5"], example["summary_bart"], example["summary_gpt2"])

        input_ids = tokenizer(example["input_text"], return_tensors="pt").input_ids.to(device)  # Move to device

        # Generate summary
        summary_ids = model.generate(input_ids)

        # Decode summary and reference
        predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        reference_summary = example["highlights"]
        print(predicted_summary)
        all_predictions.append(predicted_summary)
        all_references.append(reference_summary)

    # Calculate ROUGE scores
    result = rouge_metric.compute(predictions=all_predictions, references=all_references)

    return result


# Function to generate summaries and calculate ROUGE scores
def evaluate_model2(model, tokenizer, dataset):
    rouge_metric = load_metric("rouge")

    all_predictions = []
    all_references = []

    for example in tqdm(dataset):
        input_text = format_input(example["summary_t5"], example["summary_bart"], example["summary_gpt2"])

        input_ids = tokenizer(example["input_text"], return_tensors="pt").input_ids.to(device)  # Move to device

        # Generate summary
        summary_ids = model.generate(input_ids)

        # Decode summary and reference
        predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        reference_summary = example["highlights"]
        print(predicted_summary)
        #import pdb; pdb.set_trace()
        all_predictions.append(predicted_summary)
        all_references.append(reference_summary)

    # Calculate ROUGE scores
    result = rouge_metric.compute(predictions=all_predictions, references=all_references)

    return result

# Evaluate normal T5
normal_t5_results = evaluate_model(t5_model, t5_tokenizer, dataset)

# Evaluate Baseline fine-tuned T5
baseline_results = evaluate_model2(fine_tuned_baseline_model, t5_tokenizer_baseline, dataset)

# Evaluate fine-tuned T5
fine_tuned_t5_results = evaluate_model2(fine_tuned_model, t5_tokenizer_fine, dataset)


# Print results
print("Normal T5 Results:")
print(normal_t5_results)

print("\nFine-tuned T5 Results:")
print(baseline_results)

print("\nFine-tuned T5 Results:")
print(fine_tuned_t5_results)



IndexError: list index out of range

In [ ]:
from datasets import load_dataset, Dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from tqdm import tqdm

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the CNN/DailyMail dataset
dataset_full = load_dataset("cnn_dailymail", "3.0.0", split="test")

# Start from index 2000 and take the next 100 samples
start_idx = 2000
end_idx = 2100
dataset = dataset_full.select(range(start_idx, end_idx))

# Load fine-tuned T5 model and tokenizer
model_path_fine = "/content/drive/MyDrive/PhD/Research/Lmixer/t5_finetuned"
t5_tokenizer_fine = T5Tokenizer.from_pretrained(model_path_fine)
fine_tuned_model = T5ForConditionalGeneration.from_pretrained(model_path_fine).to(device)

# Load baseline fine-tuned model
model_path_baseline = "/content/drive/MyDrive/PhD/Research/Lmixer/finetuned_baseline"
t5_tokenizer_baseline = T5Tokenizer.from_pretrained(model_path_baseline)
fine_tuned_baseline_model = T5ForConditionalGeneration.from_pretrained(model_path_baseline).to(device)

# Load pre-trained T5 model
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Function to evaluate models
def evaluate_model(model, tokenizer, dataset):
    rouge_metric = load_metric("rouge")

    all_predictions = []
    all_references = []

    for example in tqdm(dataset):
        input_text = example["article"]  # Use article as input
        reference_summary = example["highlights"]  # Use highlights as reference

        # Tokenize and move to device
        input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).input_ids.to(device)

        # Generate summary
        summary_ids = model.generate(input_ids, max_length=150)
        predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        print(predicted_summary)  # Debugging: print generated summaries
        all_predictions.append(predicted_summary)
        all_references.append(reference_summary)

    # Calculate ROUGE scores
    result = rouge_metric.compute(predictions=all_predictions, references=all_references)
    return result

# Evaluate normal T5
normal_t5_results = evaluate_model(t5_model, t5_tokenizer, dataset)

# Evaluate Baseline fine-tuned T5
#baseline_results = evaluate_model(fine_tuned_baseline_model, t5_tokenizer_baseline, dataset)

# Evaluate fine-tuned T5
fine_tuned_t5_results = evaluate_model(fine_tuned_model, t5_tokenizer_fine, dataset)

# Print results
print("\nNormal T5 Results:")
print(normal_t5_results)

#print("\nBaseline Fine-tuned T5 Results:")
#print(baseline_results)

print("\nFine-tuned T5 Results:")
print(fine_tuned_t5_results)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

  1%|          | 1/100 [00:01<02:17,  1.39s/it]

...

 99%|█████████▉| 99/100 [01:05<00:00,  1.79it/s]

have a number of academy players at Manchester City. manuel Pellegrini says the academy players at the club are not ready for the first team. the defending league champions also have a number of homegrown players.


100%|██████████| 100/100 [01:05<00:00,  1.52it/s]

Wigan drew 2-2 with Fulham at Carven Cottage on friday. a point lifted the latics above Millwall on goal difference. but both teams are seven points adrift of safety.



Normal T5 Results:
{'rouge1': AggregateScore(low=Score(precision=0.36401450264637103, recall=0.3007511513337129, fmeasure=0.3230037255210028), mid=Score(precision=0.3904556841202049, recall=0.3213481522893735, fmeasure=0.3443523510193595), high=Score(precision=0.4191190978798724, recall=0.34488286326117745, fmeasure=0.36729160796137594)), 'rouge2': AggregateScore(low=Score(precision=0.1392730830253169, recall=0.11464285473728861, fmeasure=0.12387642183953912), mid=Score(precision=0.15865768787286938, recall=0.13105667868639345, fmeasure=0.14052628299646136), high=Score(precision=0.1822609094727644, recall=0.14954383464805485, fmeasure=0.1600940731150025)), 'rougeL': AggregateScore(low=Score(precision=0.24093733845620788, recall=0.2003161010179672, fmeasure=0.21530458358022797), mid=Score(precision=0.25868554529288695, recall=0.2159620842507555, fmeasure=0.23040857564304834), high=Score(precision=0.27809853744815305, recall=0.2329021674825625, fmeasure=0.24643688510349113)), 'rougeLsum